In [91]:
import pandas as pd
import requests
import psycopg2
import time
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import figure


import plotly
import chart_studio
chart_studio.tools.set_credentials_file(username='thechain', api_key='Tb4kz7txNipqO2nmLwaB')
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.io as pio
import plotly.express as px

In [2]:
def parser(url):
    """Get a parsed version of an URL"""
    # Get an URL
    try:
      res = requests.get(url)
    except Exception as err:
      print('There was a problem: {}'.format(err))
    
    #Parse the URL above
    soup = BeautifulSoup(res.text)
    
    return soup

In [3]:
def get_urls(url):
    """Get the URLs of all categories on Tiki homepage"""
    categ_list = []
    
    # Parse the URL
    s = parser(url)
    
    # Take url and category then add them into List
    for i in s.find_all('a', class_='MenuItem__MenuLink-tii3xq-1 efuIbv'):
        url = i.get('href')
        category = i.find('span', class_='text').text
        categ_list.append((category, url))
        
    return categ_list
        

In [4]:
def scrape(cat, url):
    """Scrape product information of all products on a page"""
    # Initialize an empty list
    results = []
    
    # Parse
    s = parser(url)
    
    # Find all DIV TAG with class =  'product item'
    product_items = s.find_all('div', class_='product-item')
    
    # If there is no item return empty list, otherwise, take each of element from list, then take specific points in this element
    if product_items == 0:
        return []
    else:
        for product_item in product_items:
            # get data for rating and review - because they have in once tag
            rating = 0
            review = 0
            tiki_now = 0
            if len(product_item.select(".rating-content span")) > 0:
                rating = product_item.select_one(".rating-content span")["style"].split(':')[1].replace("%", "")
                if product_item.select_one(".review").text.split()[0][1:].isdigit():
                    review = product_item.select_one(".review").text.split()[0][1:]
                else:
                    review = 0
            
            if '<i class="tikicon icon-tikinow-20"></i>' in str(product_item): 
                tiki_now = 1
        
            row = [product_item['data-id'],
                   product_item['data-seller-product-id'],
                   product_item['data-title'],
                   product_item['data-price'],
                   product_item['data-brand'], #new
                   product_item.img['src'],
                   tiki_now,
                   #product_item.i['class'][0], #'tikicon' means TIKI_NOW
                   rating,
                   review,
                   cat
                   #product_item.find('p', class_='review').text.split()[0][1:] #tag p - class review - split - take the first - from position 1
                  ]
            results.append(row)
#         for product_item in product_items:
#             row = [product_item['data-id'],
#                    product_item['data-seller-product-id'],
#                    product_item['data-title'],
#                    product_item['data-price'],
#                    product_item.img['src'],
#                    cat
#                   ]
#             results.append(row)
            
    return results

In [5]:
def scrape_all():
    """Scrape all products on Tiki!"""
    print('INFO scrape_all(): Start craping')
    
    # Initialize empty 'queue' list with the value of get_urls() function
    queue = get_urls('https://tiki.vn')
    
    # Initialize empty list
    results = []
    
    # Set page = 1 as default
    page = 1
    
    # Take the last category in the list, then Pop it until there is no category 
    while len(queue) > 0:
        url = queue[len(queue)-1][1]
        cat = queue[len(queue)-1][0]
        queue.pop()
        
        print(url)
        print('Scraping', cat)
        
        # Take a list of item in 1 page
        new_rows = scrape(cat, url)
        print(len(new_rows))
        
        # if this current page has product, continue to take product on next page until no product
        if len(new_rows) > 0:
            results += new_rows
            #page = int(url[url.rfind('&page=')+6:]) + 1
            #url = url[:url.rfind('&page=')+6] + str(page)
            page += 1
            url = url.split('&page=')[0] + "&page=" + str(page)
            print('Scraping', cat, ' Page:', page)
            # The last element of this list [category, link] is always the link of current page
            queue.append([cat, url])
        else:
            page = 1
          
    return results       

In [6]:
def get_connection():
    connection = psycopg2.connect(user = "postgres",
                                  password = 'P@ssword130',
                                  host = "localhost",
                                  port = "5432",
                                  database = "postgres")
    return connection

In [7]:
def create_products_table():
    
    print('INFO create_products_table(): Create table products')
    
    try:
        # Create connection & cursor
        connection = get_connection()
        cursor = connection.cursor()
        
        # Execute the query
        cursor.execute('CREATE TABLE IF NOT EXISTS Products_RAW ('
                           'ProductID TEXT,'
                           'Seller_ID TEXT,'
                           'ProductName TEXT,'
                           'Price TEXT,'
                           'Brand TEXT,'
                           'Image TEXT,'
                           'Tiki_Now TEXT,'
                           'Rating TEXT,'
                           'Comment TEXT,'
                           'Category TEXT'
                      ')')
        
        cursor.execute('CREATE TABLE IF NOT EXISTS Users ('
                          'UserID INTEGER PRIMARY KEY'
                      ')')
        
        cursor.execute('CREATE TABLE IF NOT EXISTS Comments ('
                          'CommentID INTEGER PRIMARY KEY'
                      ')')
        # Commit the changes made to our database
        connection.commit()

    except (Exception, psycopg2.Error) as error :
        print ("ERROR create_products_table(): Error while connecting to PostgreSQL", error)
        
        # In case of error, cancel all changes made to our database during the connection
        connection.cancel()
        return
    
    finally:
    
        # Close the connection & cursor
        cursor.close()
        connection.close()

In [8]:
def insert_data_to_db(data, table_name):
    
    print('INFO insert_data_to_db(): Insert data to DB')
    
    try:
        # Create connection & cursor 
        connection = get_connection()
        cursor = connection.cursor()
        
        # For each row (product) in data (product page),...
        for row in data:
            # Execute the query with each row in list data
            cursor.execute('INSERT INTO '+table_name+' VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);', row)
            
            # Commit the changes made to our database
            connection.commit()

    except (Exception, psycopg2.Error) as error :
        print ("ERROR save_data_to_db(): Error while connecting to PostgreSQL", error)
        
        # In case of error, cancel all changes made to our database during the connection
        connection.cancel()
        
    finally:
        
        # Close the connection & cursor
        cursor.close()
        connection.close()

In [169]:
def select_data_from_db():
    
    try:
        # Create connection & cursor 
        connection = get_connection()
        cursor = connection.cursor()
        
        
        # Get all data from table 'products'
        query = ''
        conti = 1
        print('Start to Analyze TIKI ...')
#         time.sleep(3)
        
        while conti == 1:
            print('Choose your chart:')
            print('1 - Total products in each category')
            print('2 - Compare total products and TIKI NOW service applied for each category')
            print('3 - Compare total products, rating and price for each category')
            print('4 - Compare Rating and Interesting for each category')
            
            statement = int(input('Input your choose: '))
            plt.figure(figsize=(20,20))
            
            if statement == 1:
                # Prepare Data
                query = 'SELECT Category, COUNT(ProductID) AS Item FROM Products GROUP BY Category'
                cursor.execute(query)
                data = cursor.fetchall()

                # Draw Chart
                #plt.figure(figsize=(15,15)) #fix size of this chart

                df = pd.DataFrame( [[ij for ij in i] for i in data] ) #create DataFrame from result of query
                df.rename(columns={0: 'Category', 1: 'Item'}, inplace=True); #Rename column in DataFrame
                df = df.sort_values(['Item'], ascending=[1]); #Sort in DataFrame

                fig = px.bar(df, x='Category', y='Item',
                             hover_data=['Category', 'Item'], color='Item',
                             labels={'pop':'Amount of products in each category'}, height=600)
                fig.show()
                
            elif statement == 2:
                # Prepare Data
                query = 'SELECT Category, COUNT(ProductID) AS Item, SUM(Tiki_Now) AS TIKINOW FROM Products GROUP BY Category'
                cursor.execute(query)
                data = cursor.fetchall()
                

                #DataFrame1 for Item
                df = pd.DataFrame( [[ij for ij in i] for i in data] ) #create DataFrame from result of query
                df.rename(columns={0: 'Category', 1: 'Item', 2: 'TIKINOW'}, inplace=True); #Rename column in DataFrame
                df = df.sort_values(['Item'], ascending=[1]); #Sort in DataFrame

                # Draw Chart
                #plt.figure(figsize=(15,15))
                
                fig = go.Figure()
                # Add 2 bar charts together
                fig.add_trace(go.Bar(
                    x=df['Category'],
                    y=df['Item'],
                    name='Item',
                    marker_color='indigo'
                ))
                fig.add_trace(go.Bar(
                    x=df['Category'],
                    y=df['TIKINOW'],
                    name='Tiki NOW',
                    marker_color='blueviolet'
                ))

                # Here we modify the tickangle of the xaxis, resulting in rotated labels.
                fig.update_layout(barmode='group')
                fig.show()
                
            elif statement == 3:   
                # Prepare Data
                query = 'SELECT Category, COUNT(ProductID) AS Item, AVG(Rating)/20 AS Star, AVG(Price) AS Price FROM Products GROUP BY Category'
                cursor.execute(query)
                data = cursor.fetchall()
                
                # Draw Chart
                #plt.figure(figsize=(15,15)) #fix size of this chart

                df = pd.DataFrame( [[ij for ij in i] for i in data] ) #create DataFrame from result of query
                df.rename(columns={0: 'Category', 1: 'Item', 2: 'Star', 3: 'Price'}, inplace=True); #Rename column in DataFrame
                
                fig = px.scatter(df, x="Price", y="Star", 
                                 size="Item", 
                                 color="Category",
                                 hover_name="Item", 
                                 log_x=True, size_max=60)
                fig.show()
            
            elif statement == 4:
                # Prepare Data
                query = 'SELECT Category, AVG(Rating)/20 AS Star, AVG(Comment) AS Comment FROM Products WHERE Rating > 0 AND Comment > 0 GROUP BY Category'
                cursor.execute(query)
                data = cursor.fetchall()
                
                # Draw Chart
                #plt.figure(figsize=(15,15)) #fix size of this chart

                df = pd.DataFrame( [[ij for ij in i] for i in data] ) #create DataFrame from result of query
                df.rename(columns={0: 'Category', 1: 'Star', 2: 'Comment'}, inplace=True);
                df = df.sort_values(['Star'], ascending=[0]);
                
                fig = go.Figure()
                fig.add_trace(go.Bar(
                    y=df['Category'],
                    x=df['Star'],
                    name='Rating',
                    orientation='h',
                    marker=dict(
                        color='rgba(246, 78, 139, 0.6)',
                        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
                    )
                ))
                fig.add_trace(go.Bar(
                    y=df['Category'],
                    x=df['Comment'],
                    name='Interesting',
                    orientation='h',
                    marker=dict(
                        color='rgba(58, 71, 80, 0.6)',
                        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
                    )
                ))

                fig.update_layout(barmode='stack')
                fig.show()
            
            conti = int(input('Do you want to continue to analyze? : 1 - Yes || 0 - No, that is enough information: '))
            

    except (Exception, psycopg2.Error) as error :
        print ("ERROR save_data_to_db(): Error while connecting to PostgreSQL", error)
        
        # In case of error, cancel all changes made to our database during the connection
        connection.cancel()
        
    finally:
        
        # Close the connection & cursor
        cursor.close()
        connection.close()

In [170]:
select_data_from_db()

Start to Analyze TIKI ...
Choose your chart:
1 - Total products in each category
2 - Compare total products and TIKI NOW service applied for each category
3 - Compare total products, rating and price for each category
Input your choose: 1


Do you want to continue to analyze? : 1 - Yes || 0 - No, that is enough information: 1
Choose your chart:
1 - Total products in each category
2 - Compare total products and TIKI NOW service applied for each category
3 - Compare total products, rating and price for each category
Input your choose: 2


Do you want to continue to analyze? : 1 - Yes || 0 - No, that is enough information: 1
Choose your chart:
1 - Total products in each category
2 - Compare total products and TIKI NOW service applied for each category
3 - Compare total products, rating and price for each category
Input your choose: 3


Do you want to continue to analyze? : 1 - Yes || 0 - No, that is enough information: 1
Choose your chart:
1 - Total products in each category
2 - Compare total products and TIKI NOW service applied for each category
3 - Compare total products, rating and price for each category
Input your choose: 4


Do you want to continue to analyze? : 1 - Yes || 0 - No, that is enough information: 0


<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>